# **Prediction Model_PFRHSC** 

### Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.regularizers import l2
from tensorflow.keras.models import load_model

### Mounting our dataset

In [2]:
data = pd.read_csv('Test data.csv')
test_df = pd.DataFrame(data)
test_df.describe()

,Fiber,Temperature,Compressive Strength
count,20.000000,20.000000,20.000000
mean,0.750000,365.000000,34.876000
std,0.573539,269.307103,7.006734
min,0.000000,25.000000,22.670000
25%,0.375000,150.000000,29.867500
50%,0.750000,350.000000,35.080000
75%,1.125000,550.000000,41.175000
max,1.500000,750.000000,46.520000


In [3]:
X = test_df[['Fiber', 'Temperature']]
y = test_df[['Compressive Strength']]
print(X.shape)
print(y.shape)

(20, 2)
(20, 1)


In [4]:
y

,Compressive Strength
0,41.31
1,35.96
2,30.31
3,28.54
4,22.67
5,43.56
6,38.65
7,32.72
8,31.35
9,24.51


### Splitting the dataset

In [5]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size = 0.4, random_state = 1)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size = 0.5, random_state = 1)

In [6]:
print(X_train.shape)
print(X_cv.shape)
print(X_test.shape)
print(y_train.shape)

(12, 2)
(4, 2)
(4, 2)
(12, 1)


### Preparing the data for cross-validation

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cv_scaled = scaler.transform(X_cv)
X_test_scaled = scaler.transform(X_test)

### Building different architecture for our model

In [8]:
def ANN_model(reg):
    tf.random.set_seed(20)
    
    model_1 = Sequential(
        [
            Dense(3, activation='relu', kernel_regularizer=l2(reg)),
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear')
        ],
        name='model_1'
    )

    model_2 = Sequential(
        [
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(2, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear'),
        ],
        name='model_2'
    )

    model_3 = Sequential(
        [
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(3, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear'),
        ],
        name='model_3'
    )
    
    model_4 = Sequential(
        [
            Dense(5, activation='relu', kernel_regularizer=l2(reg)),
            Dense(3, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear'),
        ],
        name='model_4'
    )
    
    model_list = [model_1, model_2, model_3, model_4]
    
    return model_list
    

In [9]:
def train_and_evaluate_model(model, X_train, y_train, X_cv, y_cv, model_identity):
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.1)
    )

    print(f"Model's name: {model.name}..." + 
         f"With Regularization: {reg}")
    model.fit(
        X_train, y_train,
        epochs=100,
        verbose=0
    )
    
    model.save(f"{model_name}.keras")
    
    yhat_train = model.predict(X_train)
    train_mse = mean_squared_error(y_train, yhat_train) / 2
    print(f"Training data MSE: {train_mse}")

    yhat_cv = model.predict(X_cv)
    cv_mse = mean_squared_error(y_cv, yhat_cv) / 2
    print(f"CV data MSE: {cv_mse}")

    r2 = r2_score(y_cv, yhat_cv)
    print(f"R2 value: {r2}\n\n")

regularization_strengths = [0.01, 0.1, 0.001]
saved_models = {}

for reg in regularization_strengths:
    models_with_reg = ANN_model(reg)

    
    for i, model in enumerate(models_with_reg):
        model_name = f"model_{i + 1}_reg_{reg}"
        train_and_evaluate_model(model, X_train_scaled, y_train, X_cv_scaled, y_cv, model_name)
        
        saved_models[model_name] = load_model(f"{model_name}.keras")


Model's name: model_1...With Regularization: 0.01
1/1 [==============================] - 0s 62ms/step
Training data MSE: 0.860116155868825
1/1 [==============================] - 0s 15ms/step
CV data MSE: 0.7413630639806563
R2 value: 0.9730636023874578


Model's name: model_2...With Regularization: 0.01
1/1 [==============================] - 0s 31ms/step
Training data MSE: 1.7104887480243516
1/1 [==============================] - 0s 11ms/step
CV data MSE: 1.9409605129355587
R2 value: 0.9294778946148843


Model's name: model_3...With Regularization: 0.01
1/1 [==============================] - 0s 35ms/step
Training data MSE: 1.0642356968431648
1/1 [==============================] - 0s 17ms/step
CV data MSE: 2.467339123431353
R2 value: 0.9103526586327715


Model's name: model_4...With Regularization: 0.01
1/1 [==============================] - 0s 35ms/step
Training data MSE: 0.9437928371444834
1/1 [==============================] - 0s 14ms/step
CV data MSE: 1.3170252694494369
R2 value: 0.9

In [10]:
saved_models

{'model_1_reg_0.01': <keras.src.engine.sequential.Sequential at 0x2081b736b90>,
 'model_2_reg_0.01': <keras.src.engine.sequential.Sequential at 0x2081b775290>,
 'model_3_reg_0.01': <keras.src.engine.sequential.Sequential at 0x2081a402490>,
 'model_4_reg_0.01': <keras.src.engine.sequential.Sequential at 0x2081c80a350>,
 'model_1_reg_0.1': <keras.src.engine.sequential.Sequential at 0x2081f383990>,
 'model_2_reg_0.1': <keras.src.engine.sequential.Sequential at 0x2081f59b190>,
 'model_3_reg_0.1': <keras.src.engine.sequential.Sequential at 0x2081f716b90>,
 'model_4_reg_0.1': <keras.src.engine.sequential.Sequential at 0x208207da150>,
 'model_1_reg_0.001': <keras.src.engine.sequential.Sequential at 0x2082077aed0>,
 'model_2_reg_0.001': <keras.src.engine.sequential.Sequential at 0x20821b06490>,
 'model_3_reg_0.001': <keras.src.engine.sequential.Sequential at 0x20821b38490>,
 'model_4_reg_0.001': <keras.src.engine.sequential.Sequential at 0x20822d4f910>}

In [11]:
y_test

,Compressive Strength
17,36.15
6,38.65
16,41.54
4,22.67


### Based on the trial results, "model_4_reg_0.001" is selected

In [21]:
selected_model = saved_models['model_4_reg_0.001']

print(f"Selected Model: {selected_model.name}")
yhat_test = selected_model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, yhat_test) / 2
print(f"Mean Squared Error (MSE) for test data:{test_mse}")
test_mae = mean_absolute_error(y_test, yhat_test)
print(f"Mean Absolute Error (MAE) for test data:{test_mae}")
test_r2 = r2_score(y_test, yhat_test)
print(f"R2 for test data:{test_r2}")
test_rmse = np.sqrt(test_mse)
print(f"Root Mean Squared Error (RMSE) for test data:{test_rmse}")

Selected Model: model_4
1/1 [==============================] - 0s 29ms/step
Mean Squared Error (MSE) for test data:1.276406596818067
Mean Absolute Error (MAE) for test data:1.477913436889649
R2 for test data:0.951189151102336
Root Mean Squared Error (RMSE) for test data:1.1297816589138217


In [29]:
print(f"Original: \n {y_test['Compressive Strength']}" + 
      f"\nPrediction: \n{yhat_test}")

Original: 
 17    36.15
6     38.65
16    41.54
4     22.67
Name: Compressive Strength, dtype: float64
Prediction: 
[[38.46475 ]
 [39.65153 ]
 [43.32868 ]
 [23.476696]]


### Combining all the data back again for the final prediction

In [16]:
combined_X = np.vstack([X_train_scaled, X_cv_scaled, X_test_scaled])
combined_y = np.concatenate([y_train, y_cv, y_test])

In [22]:
selected_model = saved_models['model_4_reg_0.001']

print(f"Selected Model: {selected_model.name}")
yhat = selected_model.predict(combined_X)
mse = mean_squared_error(combined_y, yhat) / 2
print(f"Mean Squared Error (MSE):{mse}")
mae = mean_absolute_error(combined_y, yhat)
print(f"Mean Absolute Error (MAE):{mae}")
r2 = r2_score(combined_y, yhat)
print(f"R2:{r2}")
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE):{rmse}")

Selected Model: model_4
1/1 [==============================] - 0s 13ms/step
Mean Squared Error (MSE):0.9746706134066467
Mean Absolute Error (MAE):1.185376750946045
R2:0.958204164280355
Root Mean Squared Error (RMSE):0.9872540774322721


### Final outputs

In [31]:
original_series = pd.Series(combined_y.flatten(), name="Original")
prediction_series = pd.Series(yhat.flatten(), name="Prediction")

df_ANN = pd.concat([original_series, prediction_series], axis=1)

df_ANN

,Original,Prediction
0,32.72,34.787601
1,35.96,37.812962
2,33.96,31.762243
3,41.31,40.852917
4,26.89,28.736883
5,34.41,33.600815
6,24.51,25.059738
7,46.52,46.368633
8,31.35,29.923670
9,35.75,36.626175


In [25]:
df_ANN.to_csv('ANN_original_VS_predictions.csv', index=False)

In [27]:
model_weights = selected_model.get_weights()

for i, layer_weights in enumerate(model_weights):
    print(f"Layer {i + 1} Weights:")
    print(layer_weights)

Layer 1 Weights:
[[-1.0399208  -0.32537225 -0.33864146  0.15213986  1.1315218 ]
 [-0.38539988 -0.10660504  0.00169763 -1.4032701  -0.31934443]]
Layer 2 Weights:
[ 2.5055122 -1.4477426  0.8790605  2.2561283  3.279878 ]
Layer 3 Weights:
[[-0.3837964  -0.903822    1.578001  ]
 [-0.7696218  -0.64475775  0.76885295]
 [-1.3445897  -0.622217    0.4252068 ]
 [ 0.06644482 -0.49193752  1.5332302 ]
 [-0.94960433 -0.47104317  2.2873092 ]]
Layer 4 Weights:
[-0.6005179  -0.69031274  2.733751  ]
Layer 5 Weights:
[[0.0738318 ]
 [0.39816323]
 [1.8404794 ]]
Layer 6 Weights:
[2.6394083]
